# 04 Forecast

It is recommended to use SQL warehouse serverless to execute this notebook. Please refer to the [requirements](https://docs.databricks.com/gcp/en/sql/language-manual/functions/ai_forecast#requirement) of `AI_FORECAST`.

In [0]:
CREATE OR REPLACE TEMPORARY VIEW dbu_cost AS (
WITH
usage AS (
    SELECT usage_date, sku_name, usage_unit, usage_quantity
    FROM system.billing.usage
    WHERE usage_date >= '2025-01-01'
),
prices AS (
    SELECT
        price_start_time,
        COALESCE(price_end_time, DATE_ADD(CURRENT_DATE, 1)) AS price_end_time,
        sku_name,
        usage_unit,
        pricing.effective_list.default AS pricing
    FROM system.billing.list_prices
    WHERE currency_code = 'USD'
)
SELECT u.usage_date, 'DBU' AS sku, SUM(u.usage_quantity*p.pricing) AS cost
FROM usage AS u
LEFT JOIN prices AS p
    ON u.sku_name = p.sku_name
    AND u.usage_unit = p.usage_unit
    AND u.usage_date BETWEEN p.price_start_time AND p.price_end_time
GROUP BY 1
);

SELECT * FROM dbu_cost;

In [0]:
CREATE OR REPLACE TEMPORARY VIEW predict AS (
WITH
azure_cost AS (
    SELECT usage_date, FIRST(meter_category) AS sku, SUM(cost) AS cost
    FROM main.billing.azure_cost
    WHERE meter_category = 'Virtual Machines'
    AND usage_date >= '2025-01-01'
    GROUP BY 1
)
SELECT * FROM azure_cost
UNION ALL
SELECT * FROM dbu_cost
);

SELECT * FROM predict;

In [0]:
CREATE OR REPLACE TABLE main.billing.forecast AS
WITH predict_with_max AS (
    SELECT *, MAX(cost) OVER (PARTITION BY sku) AS max_cost FROM predict
),
predict_with_params AS (
    SELECT
        usage_date,
        sku,
        cost,
        '{"global_floor": 0, "global_cap": ' || (max_cost * 2) || '}' AS parameters
    FROM predict_with_max
)
SELECT * FROM AI_FORECAST(
    TABLE(predict_with_params),
    horizon => (CURRENT_DATE + INTERVAL 365 DAYS),
    time_col => 'usage_date',
    value_col => 'cost',
    group_col => 'sku',
    prediction_interval_width => 0.9,
    frequency => 'D',
    seed => 666,
    parameters => 'parameters'
);

SELECT * FROM main.billing.forecast